In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [10]:
r.set('foo', 'bar')

True

In [14]:
r.get('foo')

b'bar'

In [12]:
r.keys()

[b'foo']

In [17]:
print(r.get('test'))

None


In [36]:
import socket
import time
HOST = '127.0.0.1'
PORT = 60002

PACKET = [
    504,                        # BOARDLOC
    -40, -41, -42, -43,          # HVMON (0 to -80V)
    32767, 32768, 32769, 32770, # HVIMON ((65535-HVIMON) * 38.1nA) (0 to 2.5mA)
    -44,                        # RAWHVMON (0 to -80V)
    32771,                      # V12MON (19.07uV/LSB) (1.2V supply)
    32772,                      # V18MON (19.07uV/LSB) (1.8V supply)
    32773,                      # V33MON (38.10uV/LSB) (3.3V supply)
    32774,                      # V37MON (38.10uV/LSB) (3.7V supply)
    32775,                      # I10MON (182uA/LSB) (1.0V supply)
    32776,                      # I18MON (37.8uA/LSB) (1.8V supply)
    32777,                      # I33MON (37.8uA/LSB) (3.3V supply)
    320,                        # TEMP1 (0.0625*N)
    38121,                      # TEMP2 (N/130.04-273.15)
    32778,                      # VCCINT (N*3/65536)
    32779,                      # VCCAUX (N*3/65536)
    1,0,0,0,                    # UID
    3,                          # SHUTTER and LIGHT_SENSOR STATUS
    0,                          # unused
    0,0,0,0                     # FWID0 and FWID1
]

signed = [
    0,                        # BOARDLOC
    1, 1, 1, 1,          # HVMON (0 to -80V)
    0, 0, 0, 0, # HVIMON ((65535-HVIMON) * 38.1nA) (0 to 2.5mA)
    1,                        # RAWHVMON (0 to -80V)
    0,                      # V12MON (19.07uV/LSB) (1.2V supply)
    0,                      # V18MON (19.07uV/LSB) (1.8V supply)
    0,                      # V33MON (38.10uV/LSB) (3.3V supply)
    0,                      # V37MON (38.10uV/LSB) (3.7V supply)
    0,                      # I10MON (182uA/LSB) (1.0V supply)
    0,                      # I18MON (37.8uA/LSB) (1.8V supply)
    0,                      # I33MON (37.8uA/LSB) (3.3V supply)
    1,                        # TEMP1 (0.0625*N)
    1,                      # TEMP2 (N/130.04-273.15)
    0,                      # VCCINT (N*3/65536)
    0,                      # VCCAUX (N*3/65536)
    0,0,0,0,                    # UID
    0,                        # SHUTTER and LIGHT_SENSOR STATUS
    0,                        # unused
    0,0,0,0                     # FWID0 and FWID1
]

packet1 = b'\x20\xaa'

packet2 = b'\x20\x00'

packet = packet1


for i, sign in zip(PACKET, signed):
    packet += i.to_bytes(2, 'little', signed=sign)
    
def updatePacket(template):
    val = packet2
    for i, sign in zip(PACKET, signed):
        val += i.to_bytes(2, 'little', signed=sign)
    return val

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
    sock.sendto(packet, (HOST,PORT))
    time.sleep(4)
    
    for i in range(NPACKETS-2):
        packet = updatePacket(PACKET)
        sock.sendto(packet, (HOST,PORT))
        time.sleep(4)
    
    packet = updatePacket(PACKET)
    sock.sendto(packet, (HOST,PORT))

[43552, 504, -40, -41, -42, 65493, 32767, 32768, 32769, -32766, 65492, 32771, 32772, 32773, 32774, 32775, 32776, 32777, 320, 38121, 32778, 32779, 1, 0, 0, 0, 192, 0, 0, 0, 0]


In [24]:
import socket
import redis
from signal import signal, SIGINT
from sys import exit

HOST = '127.0.0.1'
PORT = 60002
NPACKETS = 5

signed = [
    0,                        # BOARDLOC
    1, 1, 1, 1,          # HVMON (0 to -80V)
    0, 0, 0, 0, # HVIMON ((65535-HVIMON) * 38.1nA) (0 to 2.5mA)
    1,                        # RAWHVMON (0 to -80V)
    0,                      # V12MON (19.07uV/LSB) (1.2V supply)
    0,                      # V18MON (19.07uV/LSB) (1.8V supply)
    0,                      # V33MON (38.10uV/LSB) (3.3V supply)
    0,                      # V37MON (38.10uV/LSB) (3.7V supply)
    0,                      # I10MON (182uA/LSB) (1.0V supply)
    0,                      # I18MON (37.8uA/LSB) (1.8V supply)
    0,                      # I33MON (37.8uA/LSB) (3.3V supply)
    1,                        # TEMP1 (0.0625*N)
    0,                      # TEMP2 (N/130.04-273.15)
    0,                      # VCCINT (N*3/65536)
    0,                      # VCCAUX (N*3/65536)
    0,0,0,0,                    # UID
    0,                        # SHUTTER and LIGHT_SENSOR STATUS
    0,                        # unused
    0,0,0,0                     # FWID0 and FWID1
]

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

r = redis.Redis(host='localhost', port=6379, db=0)

def handler(signal_recieved, frame):
    print('SIGINT or CTRL-C detected. Exiting')
    sock.close()
    exit(0)
    
def getUID(intArr):
    b = b''
    for i in intArr[::-1]:
        b += i.to_bytes(2, byteorder='big')
    return b.hex()
    
def storeInRedis(packet):
    array = []
    
    if int.from_bytes(packet[0:1], byteorder='little') != 0x20:
        return False
    if int.from_bytes(packet[1:2], byteorder='little') == 0xaa:
        r.set('StartUp', 1)
    else:
        r.set('StartUp', 0)
        
    for i, sign in zip(range(2,len(packet), 2), signed):
        array.append(int.from_bytes(packet[i:i+2], byteorder='little', signed=sign))
    
    r.set('BOARDLOC', array[0])
    r.set('HVMON0', array[1]*1.22/1000)
    r.set('HVMON1', array[2]*1.22/1000)
    r.set('HVMON2', array[3]*1.22/1000)
    r.set('HVMON3', array[4]*1.22/1000)
    
    r.set('HVIMON0', (65535-array[5])*38.1/1000000)
    r.set('HVIMON1', (65535-array[6])*38.1/1000000)
    r.set('HVIMON2', (65535-array[7])*38.1/1000000)
    r.set('HVIMON3', (65535-array[8])*38.1/1000000)
    
    r.set('RAWHVMON', array[9]*1.22/1000)
    
    r.set('V12MON', array[10]*19.07/1000000000)
    r.set('V18MON', array[11]*19.07/1000000000)
    r.set('V33MON', array[12]*38.1/1000000000)
    r.set('V37MON', array[13]*38.1/1000000000)
    
    r.set('I10MON', array[14]*182/1000000)
    r.set('I18MON', array[15]*37.8/1000000)
    r.set('I33MON', array[16]*37.8/1000000)
    
    r.set('TEMP1', array[17]*0.0625)
    r.set('TEMP2', array[18]*0.0625)
    
    r.set('VCCINT', array[19]*3/65536)
    r.set('VCCAUX', array[20]*3/65536)
     
    r.set('UID', getUID(array[21:25]))#TODO NEED TO CHANGE
    
    r.set('SHUTTER_STATUS', array[25]&0x01)
    r.set('LIGHT_SENSOR_STATUS', array[25]&0x02 >> 1)
    
    r.set('FWID0', array[27])
    r.set('FWID0', array[28])
    r.set('FWID0', array[29])
    r.set('FWID0', array[30])
    
    
signal(SIGINT, handler)

print('Running')
sock.bind((HOST,PORT))
while(True):
    packet = sock.recvfrom(64)

Running
None
SIGINT or CTRL-C detected. Exiting


SystemExit: 0

In [11]:
b = bytearray("test", encoding="utf-8")
print(b)

bytearray(b'test')


In [26]:
r.get('LIGHT_SENSOR_STATUS')

b'1'

SyntaxError: invalid syntax (<ipython-input-37-305f445fb509>, line 1)

In [98]:
i = 192#63489
ib = i.to_bytes(2, 'little')
print(ib)

b'\xc0\x00'


In [65]:
int.from_bytes(ib, 'little',signed=True)

-40

b'()*+'